In [1]:
import glob, sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
import re
import glob
from tensorflow.keras import backend as K
import tensorflow_probability as tfp
from sklearn.metrics import accuracy_score
fft_size = 16777216
time_res = 64e-6 # in seconds
T_obs = (fft_size*time_res)/60 # in minutes is equal to 17.895 minutes
freq_axis = np.fft.rfftfreq(fft_size, d=64e-6)
freq_res = 1/(T_obs*60)

2023-06-08 18:36:22.707455: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
cur_dir = ''
run = 'runBB'
X_train = np.load(cur_dir + f'raw_data/{run}/train_data_classifier.npy').astype(np.float64)
X_test = np.load(cur_dir + f'raw_data/{run}/test_data_classifier.npy').astype(np.float64)
X_val = np.load(cur_dir + f'raw_data/{run}/val_data_classifier.npy').astype(np.float64)
X_train = X_train/np.max(X_train,axis=1)[:,None]
X_test = X_test/np.max(X_test,axis=1)[:,None]
X_val = X_val/np.max(X_val,axis=1)[:,None]

Y_train = np.load(cur_dir + f'raw_data/{run}/train_labels_classifier.npy').astype(np.float64)
Y_test = np.load(cur_dir + f'raw_data/{run}/test_labels_classifier.npy').astype(np.float64)
Y_val = np.load(cur_dir + f'raw_data/{run}/val_labels_classifier.npy').astype(np.float64)


X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1) 

Y_train = (Y_train.reshape(Y_train.shape[0],1))
Y_val = (Y_val.reshape(Y_val.shape[0],1))
Y_test = (Y_test.reshape(Y_test.shape[0],1))

In [4]:
test_indices = np.load(cur_dir + f'raw_data/{run}/test_indices.npy')
labels_df = pd.read_csv(cur_dir + f'meta_data/labels_{run}.csv')

In [12]:
def period_accuracy(y_true, y_pred):
    # Ensure the predictions are the same shape as the true values
    assert y_true.shape == y_pred.shape
    return accuracy_score(y_true, y_pred)

In [9]:
# Load the best model after training
best_model = tf.keras.models.load_model("models/chunk_classify_cnn_1112973_checkpoint.h5")

In [10]:
Y_pred = best_model.predict(X_test)

2023-06-07 14:40:54.894882: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-06-07 14:40:54.895782: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2400000000 Hz


In [14]:
Y_pred.shape

(800, 2)

In [16]:
Y_pred = np.argmax(Y_pred, axis=-1)

In [22]:
Y_pred = Y_pred.reshape(Y_pred.shape[0],1)

In [23]:
accuracy = period_accuracy(Y_test, Y_pred)
print("Period accuracy: ", accuracy)

Period accuracy:  1.0
